## Chat

Sample notebook to showcase chat usage.

Required imports:

In [1]:
# Copyright (c) 2024 Microsoft Corporation.

import json
import logging
import os

from fnllm import LLMEventsLogger
from fnllm.caching import FileCache
from fnllm.openai import AzureOpenAIConfig, OpenAIChatRole, create_openai_chat_llm
from pydantic import BaseModel, Field

Example logger to showcase event logging:

In [2]:
def _create_logger():
    logger = logging.getLogger("CHAT")
    logger.setLevel(logging.DEBUG)

    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(
        logging.Formatter("[%(levelname)s %(asctime)s %(name)s] - %(message)s")
    )
    logger.handlers = [handler]

    return logger

Creating chat client:

In [3]:
config = AzureOpenAIConfig(
    endpoint=os.environ("OPENAI_API_ENDPOINT"),
    api_version=os.environ("OPENAI_API_VERSION"),
    model=os.environ("OPENAI_API_MODEL"),
    encoding=os.environ("OPENAI_ENCODING"),
)

chat_client = create_openai_chat_llm(
    config,
    events=LLMEventsLogger(_create_logger()),
    cache=FileCache("sample_chat_cache"),
)

In [4]:
system_message = OpenAIChatRole.System.message(
    "You are a coding expert that is supposed to write the code blocks that have been asked. You should also insert comments in the code to explain its operation.",
)

response_0 = await chat_client(
    "Can you write me a hello world in rust?", history=[system_message]
)

print(response_0.model_dump_json(indent=2))

[INFO 2024-08-08 09:27:07,121 CHAT] - limit acquired for request, request_tokens=52, post_request_tokens=0
[INFO 2024-08-08 09:27:07,134 CHAT] - cache hit for key=chat_ee131b2194d2b5b5b5f0d91e97cec349147d6ce5879578d56eeae7c3d61f5e3f_v2 and name=None
[INFO 2024-08-08 09:27:07,135 CHAT] - LLM usage with 238 total tokens (input=52, output=186)
[INFO 2024-08-08 09:27:07,136 CHAT] - limit released for request, request_tokens=52, post_request_tokens=0
[INFO 2024-08-08 09:27:07,137 CHAT] - post request limiting triggered, acquired extra 186 tokens
[INFO 2024-08-08 09:27:07,137 CHAT] - request succeed with 0 retries in 0.02s and used 238 tokens


{
  "output": {
    "raw_input": {
      "content": "Can you write me a hello world in rust?",
      "role": "user"
    },
    "raw_output": {
      "content": "Certainly! Here is a \"Hello, World!\" program written in Rust:\n\n```rust\n// The main function is the entry point of a Rust program\nfn main() {\n    // The println! macro prints text to the console\n    println!(\"Hello, world!\");\n}\n```\n\n### Explanation:\n\n1. `fn main() { ... }`\n   - This defines the `main` function, which is the entry point of a Rust program. Every Rust program starts execution from the `main` function.\n\n2. `println!(\"Hello, world!\");`\n   - `println!` is a macro (denoted by the `!` at the end) that prints text to the console.\n   - `\"Hello, world!\"` is the string that will be printed.\n\nThis is one of the simplest programs you can write in Rust, and it demonstrates the basic structure of a Rust program and how to produce output.",
      "role": "assistant",
      "function_call": null,
      

In [5]:
print(response_0.output.content)

Certainly! Here is a "Hello, World!" program written in Rust:

```rust
// The main function is the entry point of a Rust program
fn main() {
    // The println! macro prints text to the console
    println!("Hello, world!");
}
```

### Explanation:

1. `fn main() { ... }`
   - This defines the `main` function, which is the entry point of a Rust program. Every Rust program starts execution from the `main` function.

2. `println!("Hello, world!");`
   - `println!` is a macro (denoted by the `!` at the end) that prints text to the console.
   - `"Hello, world!"` is the string that will be printed.

This is one of the simplest programs you can write in Rust, and it demonstrates the basic structure of a Rust program and how to produce output.


Adding history from previous request:

In [6]:
response_1 = await chat_client(
    "Awesome! Can you now translate this to $language?",
    history=response_0.history,
    variables={"language": "python"},
)

print(response_1.model_dump_json(indent=2))

[INFO 2024-08-08 09:27:07,154 CHAT] - limit acquired for request, request_tokens=293, post_request_tokens=0
[INFO 2024-08-08 09:27:07,161 CHAT] - cache hit for key=chat_326150e9d533975559a19774e7fc2b7620e8e44bc98002523b1c59ab64dacf75_v2 and name=None
[INFO 2024-08-08 09:27:07,161 CHAT] - LLM usage with 504 total tokens (input=256, output=248)
[INFO 2024-08-08 09:27:07,162 CHAT] - limit released for request, request_tokens=293, post_request_tokens=0
[INFO 2024-08-08 09:27:07,162 CHAT] - post request limiting triggered, acquired extra 211 tokens
[INFO 2024-08-08 09:27:07,162 CHAT] - request succeed with 0 retries in 0.01s and used 504 tokens


{
  "output": {
    "raw_input": {
      "content": "Awesome! Can you now translate this to python?",
      "role": "user"
    },
    "raw_output": {
      "content": "Of course! Here is the equivalent \"Hello, World!\" program written in Python:\n\n```python\n# The main function is the entry point of a Python program\ndef main():\n    # The print function prints text to the console\n    print(\"Hello, world!\")\n\n# The following if statement ensures that main() is called only when this script is run directly\nif __name__ == \"__main__\":\n    main()\n```\n\n### Explanation:\n\n1. `def main():`\n   - This defines the `main` function, which acts as the entry point of the program. However, unlike Rust, Python does not require a `main` function, but it's a common convention to define one for organization.\n\n2. `print(\"Hello, world!\")`\n   - The `print` function outputs the specified string to the console.\n\n3. `if __name__ == \"__main__\":`\n   - This if-statement ensures that the `m

In [7]:
print(response_1.output.content)

Of course! Here is the equivalent "Hello, World!" program written in Python:

```python
# The main function is the entry point of a Python program
def main():
    # The print function prints text to the console
    print("Hello, world!")

# The following if statement ensures that main() is called only when this script is run directly
if __name__ == "__main__":
    main()
```

### Explanation:

1. `def main():`
   - This defines the `main` function, which acts as the entry point of the program. However, unlike Rust, Python does not require a `main` function, but it's a common convention to define one for organization.

2. `print("Hello, world!")`
   - The `print` function outputs the specified string to the console.

3. `if __name__ == "__main__":`
   - This if-statement ensures that the `main` function is called only when the script is executed directly (not when it is imported as a module in another script).
   - `main()` is then called within this if statement to ensure the program b

### JSON response

In [8]:
response_2 = await chat_client(
    """
    Can you summarize our conversation so far as a JSON object with a summary, languages and multi_platform attributes?
    """,
    history=response_1.history,
    json=True,
)

print(response_2.model_dump_json(indent=2))

[INFO 2024-08-08 09:27:07,170 CHAT] - limit acquired for request, request_tokens=617, post_request_tokens=0
[INFO 2024-08-08 09:27:07,174 CHAT] - cache hit for key=chat_5bdfb9715c1c914e469237b58adc08b5543d8786710a78e543224ce9612db330_v2 and name=None
[INFO 2024-08-08 09:27:07,175 CHAT] - LLM usage with 600 total tokens (input=536, output=64)
[INFO 2024-08-08 09:27:07,175 CHAT] - limit released for request, request_tokens=617, post_request_tokens=0
[INFO 2024-08-08 09:27:07,175 CHAT] - request succeed with 0 retries in 0.01s and used 600 tokens


{
  "output": {
    "raw_input": {
      "content": "\n    Can you summarize our conversation so far as a JSON object with a summary, languages and multi_platform attributes?\n    ",
      "role": "user"
    },
    "raw_output": {
      "content": "\n    {\n        \"summary\": \"We discussed 'Hello, World!' programs in Rust and Python. The Rust program was explained first, followed by its equivalent in Python. Both snippets were described with comments detailing their function.\",\n        \"languages\": [\"Rust\", \"Python\"],\n        \"multi_platform\": true\n    }",
      "role": "assistant",
      "function_call": null,
      "tool_calls": null
    },
    "content": "\n    {\n        \"summary\": \"We discussed 'Hello, World!' programs in Rust and Python. The Rust program was explained first, followed by its equivalent in Python. Both snippets were described with comments detailing their function.\",\n        \"languages\": [\"Rust\", \"Python\"],\n        \"multi_platform\": tru

The raw JSON response is a python dict or list:

In [9]:
print(json.dumps(response_2.raw_json, indent=4))

{
    "summary": "We discussed 'Hello, World!' programs in Rust and Python. The Rust program was explained first, followed by its equivalent in Python. Both snippets were described with comments detailing their function.",
    "languages": [
        "Rust",
        "Python"
    ],
    "multi_platform": true
}


### JSON response with model validation

In [10]:
class Summary(BaseModel):
    """Summary."""

    languages: list[str] = Field(description="List of languages covered.")

    summary: str = Field(description="Summary of what has been discussed.")

    multi_platform: bool = Field(
        description="Where the given codes can be considered multi-platform."
    )

In [11]:
response_3 = await chat_client(
    """
    Can you summarize our conversation so far returning a JSON object that follows the JSON schema below? The response is not that JSON schema, but a response that follows it.

    SCHEMA
    ------
    $schema

    YOUR RESPONSE
    -------------
    """,
    history=response_1.history,
    variables={"schema": json.dumps(Summary.model_json_schema(), indent=2)},
    json_model=Summary,
)

print(response_3.model_dump_json(indent=2))

[INFO 2024-08-08 09:27:07,189 CHAT] - limit acquired for request, request_tokens=652, post_request_tokens=0
[INFO 2024-08-08 09:27:07,194 CHAT] - cache hit for key=chat_708a50ae000e289182630f3de3021736ab3e904e492528b4d2b38f8c232c96d8_v2 and name=None
[INFO 2024-08-08 09:27:07,194 CHAT] - LLM usage with 824 total tokens (input=729, output=95)
[INFO 2024-08-08 09:27:07,195 CHAT] - limit released for request, request_tokens=652, post_request_tokens=0
[INFO 2024-08-08 09:27:07,195 CHAT] - post request limiting triggered, acquired extra 172 tokens
[INFO 2024-08-08 09:27:07,195 CHAT] - request succeed with 0 retries in 0.01s and used 824 tokens


{
  "output": {
    "raw_input": {
      "content": "\n    Can you summarize our conversation so far returning a JSON object that follows the JSON schema below? The response is not that JSON schema, but a response that follows it.\n\n    SCHEMA\n    ------\n    {\n  \"description\": \"Summary.\",\n  \"properties\": {\n    \"languages\": {\n      \"description\": \"List of languages covered.\",\n      \"items\": {\n        \"type\": \"string\"\n      },\n      \"title\": \"Languages\",\n      \"type\": \"array\"\n    },\n    \"summary\": {\n      \"description\": \"Summary of what has been discussed.\",\n      \"title\": \"Summary\",\n      \"type\": \"string\"\n    },\n    \"multi_platform\": {\n      \"description\": \"Where the given codes can be considered multi-platform.\",\n      \"title\": \"Multi Platform\",\n      \"type\": \"boolean\"\n    }\n  },\n  \"required\": [\n    \"languages\",\n    \"summary\",\n    \"multi_platform\"\n  ],\n  \"title\": \"Summary\",\n  \"type\": \"ob

The parsed JSON response is already an instance of the given model:

In [12]:
type(response_3.parsed_json)

__main__.Summary

In [13]:
if response_3.parsed_json:
    print(response_3.parsed_json.model_dump_json(indent=2))

{
  "languages": [
    "Rust",
    "Python"
  ],
  "summary": "We discussed writing a 'Hello, World!' program in Rust and then translated that program into Python. The Rust program uses the `main` function and the `println!` macro to print the message. The Python equivalent uses a `main` function and the `print` statement inside an `if __name__ == '__main__':` block.",
  "multi_platform": false
}
